In [1]:
import pandas as pd

In [2]:
df_run_v1 = pd.read_json("../reports/hyperparameter_search_gat.json")
df_run_v2 = pd.read_json("../reports/hyperparameter_search_gat_v2.json")
df_coins = pd.read_json("../data/raw/coins.json")

In [3]:
display(df_run_v1.head())
display(df_run_v2.head())

,training_loops,hidden_channels,num_layers,lr,balance_labels,accuracies
0,500,128,6,0.0005,True,"{'Chainlink': 0.739224699828473, 'Avalanche': ..."
1,4000,8,7,0.0001,True,"{'Chainlink': 0.7364391080617491, 'Avalanche':..."
2,4000,32,9,0.0001,True,"{'Chainlink': 0.729879931389365, 'Avalanche': ..."
3,2000,32,4,0.0050,True,"{'Chainlink': 0.9984905660377351, 'Avalanche':..."
4,1000,8,2,0.0001,True,"{'Chainlink': 0.9683704974271011, 'Avalanche':..."


,training_loops,hidden_channels,num_layers,lr,balance_labels,accuracies
0,4000,64,9,0.0001,True,"{'Chainlink': 0.7242950257289881, 'Avalanche':..."
1,2000,256,1,0.0001,False,"{'Chainlink': 0.99998627787307, 'Avalanche': 0..."
2,500,32,8,0.0005,True,"{'Chainlink': 0.9613447684391081, 'Avalanche':..."
3,1000,16,3,0.0005,False,"{'Chainlink': 1.0, 'Avalanche': 1.0, 'THORChai..."
4,2000,32,3,0.0001,False,"{'Chainlink': 1.0, 'Avalanche': 1.0, 'THORChai..."


In [4]:
df_coins

,name,symbol,fraud,test,max_market_cap_e9,start_date,subreddits,end_date
0,Bitcoin,BTC,False,False,1800.000,2010-07-14,"[CryptoCurrency, CryptoMoonShots, CryptoMarket...",None
1,Ethereum,ETH,False,True,550.000,2015-08-08,"[CryptoCurrency, CryptoMoonShots, CryptoMarket...",None
2,Terra Luna,LUNC,True,False,41.000,2019-07-07,"[CryptoCurrency, CryptoMoonShots, CryptoMarket...",2022-05-08
3,Avalanche,AVAX,False,False,30.000,2020-09-23,"[CryptoCurrency, CryptoMoonShots, CryptoMarket...",None
4,Cosmos,ATOM,False,True,11.000,2019-03-15,"[CryptoCurrency, CryptoMoonShots, CryptoMarket...",None
5,Chainlink,LINK,False,False,9.900,2017-10-01,"[CryptoCurrency, CryptoMoonShots, CryptoMarket...",None
6,FTX Token,FTT,True,True,9.500,2019-08-01,"[CryptoCurrency, CryptoMoonShots, CryptoMarket...",2022-11-07
7,THORChain,RUNE,False,False,1.800,2019-07-24,"[CryptoCurrency, CryptoMoonShots, CryptoMarket...",None
8,BeerCoin,BEER,True,False,0.310,2024-05-27,"[CryptoCurrency, CryptoMoonShots, CryptoMarket...",2024-06-24
9,BitForex,BF,True,False,0.057,2019-08-31,"[CryptoCurrency, CryptoMoonShots, CryptoMarket...",2024-03-05


In [5]:
def get_rating_hparams(df, df_coins):
    """Get sorted dataframe with the best hyperparameters based on the number of correct predictions (>= 0.5) and mean accuracy"""
    # Split accuracies into separate columns
    df_accuracies = pd.DataFrame(df["accuracies"].to_list())
    df = pd.concat([df, df_accuracies], axis=1)

    # Calculate mean accuracy for each coin and overall
    df["correct"] = df["accuracies"].apply(lambda x: sum([1 for acc in x.values() if acc >= 0.5]))
    df["mean_acc_non_scam"] = df[
        [coin for coin in df_coins["name"][df_coins["fraud"] == 0] if coin in df.columns]
    ].mean(axis=1)
    df["mean_acc_scam"] = df[[coin for coin in df_coins["name"][df_coins["fraud"] == 1] if coin in df.columns]].mean(
        axis=1
    )
    df["macro_acc"] = (df["mean_acc_non_scam"] + df["mean_acc_scam"]) / 2

    # Calculate the mean coin prediction
    df["mean_predictions"] = df["accuracies"].apply(
        lambda x: {
            coin: acc if coin in df_coins[df_coins["fraud"] == 1]["name"].values else 1 - acc for coin, acc in x.items()
        }
    )

    # Rating penalizes models with low mean accuracy in either class, similar to F1 score
    df["rating"] = 2 * df["mean_acc_non_scam"] * df["mean_acc_scam"] / (df["mean_acc_non_scam"] + df["mean_acc_scam"])

    return df.sort_values(by=["correct", "rating"], ascending=False)

In [6]:
get_rating_hparams(df_run_v1, df_coins).head()

,training_loops,hidden_channels,num_layers,lr,balance_labels,accuracies,Chainlink,Avalanche,THORChain,Bitcoin,Terra Luna,BeerCoin,BitForex,correct,mean_acc_non_scam,mean_acc_scam,macro_acc,mean_predictions,rating
103,4000,128,4,0.0001,True,"{'Chainlink': 0.7163636363636361, 'Avalanche':...",0.716364,0.688672,0.620445,0.705718,0.393873,0.203727,0.486021,4,0.682800,0.361207,0.522003,"{'Chainlink': 0.2836363636363639, 'Avalanche':...",0.472472
24,2000,32,3,0.0010,True,"{'Chainlink': 0.740665523156089, 'Avalanche': ...",0.740666,0.692905,0.627211,0.718914,0.326155,0.190062,0.499043,4,0.694924,0.338420,0.516672,"{'Chainlink': 0.259334476843911, 'Avalanche': ...",0.455175
40,2000,16,7,0.0005,True,"{'Chainlink': 0.719821612349914, 'Avalanche': ...",0.719822,0.747036,0.641860,0.777122,0.316480,0.185093,0.454232,4,0.721460,0.318602,0.520031,"{'Chainlink': 0.28017838765008596, 'Avalanche'...",0.442009
55,4000,128,2,0.0010,True,"{'Chainlink': 0.6757735849056601, 'Avalanche':...",0.675774,0.682813,0.641967,0.661382,0.305655,0.187578,0.489085,4,0.665484,0.327439,0.496461,"{'Chainlink': 0.3242264150943399, 'Avalanche':...",0.438917
51,4000,16,6,0.0001,True,"{'Chainlink': 0.749941680960548, 'Avalanche': ...",0.749942,0.712892,0.644044,0.753994,0.311528,0.173913,0.457679,4,0.715218,0.314373,0.514796,"{'Chainlink': 0.25005831903945197, 'Avalanche'...",0.436767


In [7]:
get_rating_hparams(df_run_v1, df_coins).mean_predictions[0]

{'Chainlink': 0.26077530017152695,
 'Avalanche': 0.321419561012531,
 'THORChain': 0.371350948220755,
 'Bitcoin': 0.277709389925649,
 'Terra Luna': 0.0009213405505000001,
 'BeerCoin': 0.072049689440993,
 'BitForex': 0.483339716583684}

In [8]:
get_rating_hparams(df_run_v2, df_coins).head()

,training_loops,hidden_channels,num_layers,lr,balance_labels,accuracies,Chainlink,Avalanche,THORChain,Bitcoin,Terra Luna,BeerCoin,BitForex,correct,mean_acc_non_scam,mean_acc_scam,macro_acc,mean_predictions,rating
32,4000,64,7,0.0010,True,"{'Chainlink': 0.611032590051457, 'Avalanche': ...",0.611033,0.999888,0.633497,0.761119,0.354831,0.318012,0.482191,4,0.751384,0.385011,0.568198,"{'Chainlink': 0.38896740994854295, 'Avalanche'...",0.509139
16,1000,32,3,0.0005,True,"{'Chainlink': 0.7816260720411661, 'Avalanche':...",0.781626,0.740728,0.760548,0.813369,0.280548,0.342857,0.420529,4,0.774068,0.347978,0.561023,"{'Chainlink': 0.21837392795883392, 'Avalanche'...",0.480120
11,1000,64,2,0.0005,True,"{'Chainlink': 0.7615917667238421, 'Avalanche':...",0.761592,0.776386,0.747869,0.814248,0.268916,0.370186,0.394868,4,0.775024,0.344657,0.559840,"{'Chainlink': 0.2384082332761579, 'Avalanche':...",0.477131
26,4000,8,5,0.0005,True,"{'Chainlink': 0.697550600343053, 'Avalanche': ...",0.697551,0.725815,0.664873,0.793688,0.345848,0.244720,0.445806,4,0.720482,0.345458,0.532970,"{'Chainlink': 0.30244939965694695, 'Avalanche'...",0.466999
24,4000,32,3,0.0010,True,"{'Chainlink': 0.639368782161234, 'Avalanche': ...",0.639369,0.686261,0.615385,0.745187,0.353564,0.229814,0.484489,4,0.671550,0.355956,0.513753,"{'Chainlink': 0.36063121783876595, 'Avalanche'...",0.465286


In [9]:
get_rating_hparams(df_run_v2, df_coins).mean_predictions[0]

{'Chainlink': 0.2757049742710119,
 'Avalanche': 0.2847812070753799,
 'THORChain': 0.3045493287875559,
 'Bitcoin': 0.24708960840725092,
 'Terra Luna': 0.096855925371415,
 'BeerCoin': 0.42608695652173906,
 'BitForex': 0.412868632707774}